In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as tf_keras

In [4]:
with open("Data/nietzsche.txt", "rt") as f:
    data = f.read()

In [5]:
data_lower = data.lower()
data_lower[:30]

'preface\n\n\nsupposing that truth'

In [6]:
print(set(data_lower))
sorted_chars = sorted( set(data_lower))
print(sorted_chars)
print("*" * 100)
char_to_idx = {ch:i for i, ch in enumerate(sorted_chars)}

idx_to_char = {i:ch for i, ch in enumerate(sorted_chars)}
idx_to_char2 = {i:ch for ch, i in char_to_idx.items()}
print(char_to_idx)
print("*" * 100)
print(idx_to_char)
print("*" * 100)
print(idx_to_char2)

{"'", '=', '8', '[', '(', '4', 'k', 's', 'p', 'o', 'f', '5', '"', '3', 'e', '?', '0', 'a', 'q', ' ', '.', ':', 't', '1', ']', 'r', 'h', 'v', 'l', 'ä', '7', 'æ', 'c', 'm', 'é', 'b', 'n', 'd', '2', '-', '!', ',', '6', 'i', ';', 'g', 'y', 'x', 'u', 'w', ')', 'ë', '9', '\n', 'z', '_', 'j'}
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']
****************************************************************************************************
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 

In [7]:
sequence_length = 50 #연속된 문자의 수
step = 3 # stride (3개 문자씩 이동하면서 데이터 추출)

sequences = [] # (batch 크기 : 한번에 처리하는 갯수, 입력문자갯수, 단어사전크기)
next_chars = [] #배치크기, 단어사전크기

for idx in range(0, len(data_lower) - sequence_length, step):
    sequences.append(data_lower[idx:idx+sequence_length])
    next_chars.append(data_lower[idx+sequence_length])

print(len(sequences), len(next_chars))
print(sequences[0], next_chars[0])

200281 200281
preface


supposing that truth is a woman--what th e


In [8]:
x = np.zeros(shape=(len(sequences), sequence_length, len(sorted_chars)))
y = np.zeros(shape=(len(sequences), len(sorted_chars)))

for si, sequence in enumerate(sequences): # si는 입력 문장 번호
    # print(si, sequence)
    for ci, ch in enumerate(sequence): #ci는 한개의 입력문장 안의 문자 순서 번호
        x[si, ci, char_to_idx[ch]] = 1
        y[si, char_to_idx[next_chars[si]]] = 1
        # print(ci, ch, end=", ")
        pass
    
    # if si == 2:
    #     break

In [10]:
x.shape, y.shape

((200281, 50, 57), (200281, 57))

In [19]:
x[0,0,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [23]:
loaded_model = tf_keras.models.load_model(r"C:\Work\2024\minion\Workspace\dl_basic\Model\generation-model.keras")

In [24]:
def select_character(preds, temperature=1.0): # temperature 값이 작을 수록 낮은 확률의 값이 선택 가능성이 낮짐
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)    
    probas = np.random.multinomial(1, preds, 1) # 주어진 확률에 따라 다음 값 랜덤 선택
    return np.argmax(probas)

In [35]:
start_idx = np.random.randint(0, len(data_lower) - sequence_length)
seed_text = data_lower[ start_idx: start_idx + sequence_length]
# print(seed_text)
# print("*" * 100)
for temperature in [0.5, 1.0, 1.5]:
    full_text = seed_text
    print("\n")
    print("*" * 100)
    print("temperature : {0}".format(temperature))
    print("\n")
    print(seed_text, end="")
    

    for idx in range(100):
        sample = np.zeros(shape=(1, sequence_length, len(sorted_chars))) # 1, 50, 57
        for cidx, c in enumerate(seed_text):
            sample[0, ci, char_to_idx[c]] = 1

        predicted_values = loaded_model.predict(sample, verbose=0)
        #selected_char_idx = predicted_values[0].argmax()
        selected_char_idx = select_character(predicted_values[0], 1)
        full_text = sorted_chars[selected_char_idx]
        seed_text = full_text[-50:]
        #seed_text = full_text[idx+1:]

        print(sorted_chars[selected_char_idx], end="")




****************************************************************************************************
temperature : 0.5


ich
all these things are wafted away from the souled wane ined bly), sthinthes!
bly-------ces uly? outs.
hescares, winines ined ped thiners.. ous thar

****************************************************************************************************
temperature : 1.0


r, uly,
wis outhe bly-aninemou mouly, memesthestes, out; uly an winouly------------rs g, wines
thinem

****************************************************************************************************
temperature : 1.5


mers d ouly ined anes fustheshuly,
cthes, anes mousthes, uly. inxthes ganes," ousthed, ine-s, bly, gs